K-NN Classification workbook.

Imports

In [1]:
import math
import multiprocessing
import string
import time
from collections import Counter

import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

# from scipy.sparse import csr_matrix

We need `punctuation` and `stopwords` for english language.
We also need a Stemmer to do some stemming. 

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
stemmer = nltk.PorterStemmer()
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /home/dhimal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/dhimal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


To read the training data.

Make sure the data file and its path is correct. Trim the data for testing. This will call preprocessing methods below

In [20]:
def get_training_data():
    # training file
    train_file = "../../data/1580449515_4035058_train_file.dat"
    train = pd.read_table(train_file, header=None, skip_blank_lines=False)
    train.columns = ["label", "data"]
    train = train.dropna()
    train = train.reset_index(drop=True)
    data_processed = [preprocess(line) for index, line in enumerate(train.data)]
    return data_processed, train.label

To read the test data.

Make sure the data file and its path is correct. Trim the data for testing. This will call preprocessing methods below

In [19]:
def get_raw_test_data():
    # test file
    test_file = "../../data/1580449515_4313154_test.dat"
    with open(test_file, "r") as file:
        data = file.readlines()
    return data

For Preprocessing:
   * Change the star ratings in numerical values in the document to words 
   * For each word in the document
       * lowercase the word
       * remove if the word is punctiation
       * remove it the word is a stopword
       * remove if length of the word is less than or equal to 3
       * Use the stemmer (PorterStemmer) to stem the words

In [5]:
def preprocess(document):
    # preserve any stars ratings, people usualy say I give this product 1 star, 2 star, 3 star, 4 star, and 5 star
    if "1 star" in document:
        document = document.replace("1 star", "onestar")
    if "2 star" in document:
        document = document.replace("2 star", "twostars")
    if "3 star" in document:
        document = document.replace("3 star", "threestars")
    if "4 star" in document:
        document = document.replace("4 star", "fourstars")
    if "5 star" in document:
        document = document.replace("5 star", "fivestars")

    tokens = nltk.word_tokenize(document)
    # delete any digits
    tokens = [word for word in tokens if not word.isdigit()]
    # lower case
    tokens = [word.lower() for word in tokens if type(word) == str]
    # remove punctuations
    tokens = [word for word in tokens if word not in string.punctuation]
    # remove stopwords
    tokens = [word for word in tokens if not word in stop_words]
    # remove any words less and 3 in length
    tokens = [word for word in tokens if not len(word) <= 3]
    # stemming
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

This method is to make a list of each word and the document it appears in.

In [6]:
def df_list(document, index, DF_list):
    for token in document:
        try:
            DF_list[token].add(index)
        except:
            DF_list[token] = {index}

This method is to update the df_list above with the number of documents a word appears in. We don't need to keep track of the document index.

In [7]:
def update_DF_list(word, DF_list):
    DF_list[word] = len(DF_list[word])

This method is to create a TF-IDF vector for each document.
The equations in here are taken out of the class slides.

In [8]:
def document_vector(
        document,
        total_vocabulary,
        N,
        DF_list):
    """
    Returns TF-IDF for each document
    """
    # each document need the length of total vocab
    doc_vector = np.zeros((len(total_vocabulary)))

    counter = Counter(document)
    words_count = len(document)

    for token in np.unique(document):
        tf = counter[token] / words_count
        df = doc_freq(DF_list, token)
        idf = math.log((N + 1) / (df + 1))  # why plus 1?
        try:
            ind = total_vocabulary.index(token)
            doc_vector[ind] = tf * idf
        except:
            pass
    return doc_vector

This method is to look up how many times a word appears in a document.

In [9]:
def doc_freq(DF_matrix, word):
    frequenxy = 0
    try:
        frequenxy = DF_matrix[word]
    except:
        pass
    return frequenxy

#### Multiprocessing

This method will call the label method to label the training instance for the index taken out from the task queue.

In [10]:
def process_tasks(
        task_queue,
        k,
        total_vocabulary,
        train_matrix,
        train_label,
        test_data_raw,
        DF_list,
        test_y):
    """
    This method is to support multi-processing
    """
    while not task_queue.empty():
        task = task_queue.get()
        label(
            vocabulary=total_vocabulary,
            train_data_matrix=train_matrix,
            train_label=train_label,
            test_document_index=task,  # should have the document index
            test_data_raw=test_data_raw,
            DF_list_train=DF_list,
            test_y=test_y,
            k=k)
    return True

This method will add all the tasks (labeling the test instances) to a queue at the begining of the K-NN process

In [11]:
def add_tasks(
        task_queue,
        test_data_list):
    """
    This method is to support multi-processing
    """
    for test_document_index in range(len(test_data_list)):
        task_queue.put(test_document_index)
        print(test_document_index)
    return task_queue

#### K-Nearest Neighbor

This is where the magic happens. Given:
    * TF-IDF matrix
    * Training labels
    * test instance index
    * k
This method is take the current instance and its TF-IDF vector and compute a cosine similarity (our distance measure) with all the training instances. This will be saved to a distance vector.

After sorting the distance vector, this method will select the top K neighbors and look at their labels

In [12]:
def cosine_simimarity(document_a, document_b):
    # normalized cosine similarity
    return np.dot(document_a, document_b) / (np.linalg.norm(document_a) * np.linalg.norm(document_b))

In [13]:
def label(
        vocabulary,
        train_data_matrix,  # This is a TF-IDF matrix
        train_label,
        test_document_index,
        test_data_raw,  # this is the raw test data, not pre-processed yet
        DF_list_train,
        test_y,
        k):
    test_raw = test_data_raw[test_document_index]

    test_processed = preprocess(test_raw)

    # Generate the TF_IDF for the test instance
    test_TF_IDF_vector = document_vector(document=test_processed,
                                         N=len(train_label) + 1,  # because we are adding 1 more?
                                         DF_list=DF_list_train,
                                         total_vocabulary=vocabulary)

    print("Finding label for test item ", test_document_index)

    # cosine similarity
    distances = [(index, (cosine_simimarity(test_TF_IDF_vector, doc_vector))) for index, doc_vector in
                 enumerate(train_data_matrix)]

    # get k-nearest neighbors
    # np.array(distances).sort()[::-1]
    # the closer the documents are by angle, the higher is the Cosine Similarity
    distances.sort(key=lambda x: x[1])
    neighbor_indices = distances[::-1]
    neighbor_indices = neighbor_indices[:k]

    label_list = []
    for value in neighbor_indices:
        label_list.append(train_label[value[0]])

    # get a list of labels from the neighbors and sum the list
    # labels_list = train_label[neighbor_indices].tolist()
    label_sum = sum(label_list)

    # classify based on label_sum
    if label_sum > 0:
        label_this = +1
    else:
        label_this = -1
        # print(test_data[test_document_index])
        # print("labeled as : ", label_this)

    test_y[test_document_index] = label_this

Manages multiple processes calling the label method (which is really where the K-NN happens).

Uses `multiprocessing.Array` to store the labels of the test instances. This was one of the options to share a data structures between multiple processes.



In [14]:
def k_nearest_neighbor(
        k,
        vocabulary,
        test_data_raw,
        DF_list,
        train_matrix,
        train_label):
    test_y = multiprocessing.Array('i', len(test_data_raw))

    num_cpus = multiprocessing.cpu_count() - 2
    queue = multiprocessing.Queue()
    full_task_queue = add_tasks(
        queue,
        test_data_raw)

    processes = []
    print(f'Running with {num_cpus} processes!')
    start = time.time()
    for n in range(num_cpus):
        process = multiprocessing.Process(
            target=process_tasks, args=(
                full_task_queue,
                k,
                vocabulary,
                train_matrix,
                train_label,
                test_data_raw,
                DF_list,
                test_y,))
        processes.append(process)
        process.start()
    for process in processes:
        process.join()
    print(f'Time taken = {time.time() - start:.10f}')

    review_ratings = []
    for rating in test_y:
        if rating is 1:
            review_ratings.append("+1")
        else:
            review_ratings.append("-1")
    return review_ratings

#### Let's classify

Calling of the methods (should really be in a main method)

Get the training data and the test data.
Combine them.

In [21]:
training_data, training_labels = get_training_data()
# Raw test data
raw_test_data = get_raw_test_data()

In [24]:
training_data

[['book',
  'life',
  'saver',
  'help',
  'abl',
  'back',
  'track',
  'trend',
  'answer',
  'pediatrician',
  'question',
  'commun',
  'differ',
  'time',
  'night',
  'newborn',
  'think',
  'thing',
  'everyon',
  'requir',
  'leav',
  'hospit',
  'went',
  'page',
  'newborn',
  'version',
  'move',
  'infant',
  'version',
  'finish',
  'second',
  'infant',
  'book',
  'third',
  'total',
  'right',
  'babi',
  'turn',
  'thing',
  'must',
  'have',
  'babi'],
 ['bought',
  'time',
  'older',
  'bought',
  'newborn',
  'super',
  'easi',
  'help',
  'keep',
  'track',
  'daili',
  'routin',
  'start',
  'go',
  'sitter',
  'went',
  'back',
  'work',
  'help',
  'know',
  'went',
  'better',
  'prepar',
  'even',
  'would',
  'like',
  'sick',
  'help',
  'keep',
  'track',
  'mani',
  'diaper',
  'produc',
  'make',
  'sure',
  'get',
  'dehydr',
  'note',
  'section',
  'side',
  'bottom',
  'use',
  'sitter',
  'write',
  'small',
  'note',
  'whether',
  'like',
  'lunch'

In [25]:
training_labels

0        1
1        1
2        1
3        1
4        1
        ..
18492   -1
18493   -1
18494   -1
18495   -1
18496   -1
Name: label, Length: 18497, dtype: int64

In [26]:
raw_test_data

["Perfect for new parents. We were able to keep track of baby's feeding, sleep and diaper change schedule for the first two and a half months of her life. Made life easier when the doctor would ask questions about habits because we had it all right there!\n",
 "Helps me know exactly how my babies day has gone with my mother in law watching him while I go to work.  It also has a section for her to write notes and let me know anything she may need.  I couldn't be happier with this book.\n",
 "I wanted an alternative to printing out daily log sheets for the nanny to fill out, and this has worked out great! I'm no longer searching my daughter's bag for a crumpled piece of paper each day. It's also nice to be able to look back on previous days and weeks for eating and sleeping patterns. I would have preferred a plastic-type cover, but it's held up well so far.\n",
 'My 3 month old son spend half of his days with my mother and half with a neighbor while I worked.  I had them track his activi

In [31]:
len(raw_test_data)

18506

Make a document frequency list for each word that appears in the whole dataset (training + test)

In [22]:
DF_list_training = {}
# this is going to update the DF_
[df_list(document, index, DF_list_training) for index, document in enumerate(training_data)]
# again updates the list?
[update_DF_list(word, DF_list_training) for word in DF_list_training]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [27]:
DF_list_training

{'book': 160,
 'life': 346,
 'saver': 118,
 'help': 1369,
 'abl': 1275,
 'back': 2459,
 'track': 73,
 'trend': 29,
 'answer': 60,
 'pediatrician': 39,
 'question': 64,
 'commun': 25,
 'differ': 1393,
 'time': 4087,
 'night': 1082,
 'newborn': 731,
 'think': 2001,
 'thing': 2475,
 'everyon': 235,
 'requir': 296,
 'leav': 564,
 'hospit': 248,
 'went': 709,
 'page': 64,
 'version': 283,
 'move': 976,
 'infant': 698,
 'finish': 155,
 'second': 907,
 'third': 133,
 'total': 418,
 'right': 1559,
 'babi': 6546,
 'turn': 1048,
 'must': 540,
 'have': 6,
 'bought': 2966,
 'older': 587,
 'super': 678,
 'easi': 2916,
 'keep': 2533,
 'daili': 166,
 'routin': 56,
 'start': 1586,
 'go': 1276,
 'sitter': 24,
 'work': 4149,
 'know': 1430,
 'better': 2005,
 'prepar': 79,
 'even': 2998,
 'would': 5500,
 'like': 5479,
 'sick': 62,
 'mani': 1131,
 'diaper': 1488,
 'produc': 65,
 'make': 2922,
 'sure': 1365,
 'get': 2045,
 'dehydr': 4,
 'note': 232,
 'section': 97,
 'side': 1532,
 'bottom': 1022,
 'use': 49

Get our vocabulary list from the DF_list

In [23]:
vocab_training = [word for word in DF_list_training]

In [28]:
vocab_training

['book',
 'life',
 'saver',
 'help',
 'abl',
 'back',
 'track',
 'trend',
 'answer',
 'pediatrician',
 'question',
 'commun',
 'differ',
 'time',
 'night',
 'newborn',
 'think',
 'thing',
 'everyon',
 'requir',
 'leav',
 'hospit',
 'went',
 'page',
 'version',
 'move',
 'infant',
 'finish',
 'second',
 'third',
 'total',
 'right',
 'babi',
 'turn',
 'must',
 'have',
 'bought',
 'older',
 'super',
 'easi',
 'keep',
 'daili',
 'routin',
 'start',
 'go',
 'sitter',
 'work',
 'know',
 'better',
 'prepar',
 'even',
 'would',
 'like',
 'sick',
 'mani',
 'diaper',
 'produc',
 'make',
 'sure',
 'get',
 'dehydr',
 'note',
 'section',
 'side',
 'bottom',
 'use',
 'write',
 'small',
 'whether',
 'lunch',
 'playtim',
 'includ',
 'walk',
 'etc.excel',
 'mom',
 'want',
 'kid',
 'though',
 'excel',
 'dad',
 'husband',
 'quickli',
 'forget',
 'great',
 'basic',
 'wish',
 'space',
 'bigger',
 'need',
 'struggl',
 'tri',
 'read',
 'caretak',
 'wrote',
 'togeth',
 'perfect',
 'first',
 'made',
 'feed',
 

Create a TF-IDF matrix

In [29]:
N = len(training_data)
tf_idf_matrix_training = [document_vector(
    document,
    vocab_training,
    N,
    DF_list_training) for document in training_data]

In [30]:
tf_idf_matrix_training

[array([0.22590541, 0.09466888, 0.12014987, ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.37952108, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.43127396, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0.20626146, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.23720068, 0.09940233, 0.12615736, ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.12484246, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.1757042 , 0.07363135, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ...

Sparse matrix

In [5]:
# mat_sparse = csr_matrix(tf_idf_matrix_training)
# mat_sparse.sort_indices()

K-NN with `k=25`

In [32]:
print("INFO: Starting K-NN...")
k = 25
# returns a set of results
ratings_k25 = k_nearest_neighbor(
    k=k,
    vocabulary=vocab_training,
    test_data_raw=raw_test_data,
    DF_list=DF_list_training,
    train_matrix=tf_idf_matrix_training,
    train_label=np.array(training_labels))
print("INFO: K-nearest neighbor done!")

INFO: Starting K-NN...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271

3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393
3394
3395
3396
3397
3398
3399
3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3410
3411
3412
3413
3414
3415
3416
3417
3418
3419
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3430
3431
3432
3433
3434
3435
3436
3437
3438
3439
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
3450
3451
3452
3453
3454
3455
3456
3457
3458
3459
3460
3461
3462
3463
3464
3465
3466
3467
3468
3469
3470
3471
3472
3473
3474
3475
3476
3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576


6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003


10281
10282
10283
10284
10285
10286
10287
10288
10289
10290
10291
10292
10293
10294
10295
10296
10297
10298
10299
10300
10301
10302
10303
10304
10305
10306
10307
10308
10309
10310
10311
10312
10313
10314
10315
10316
10317
10318
10319
10320
10321
10322
10323
10324
10325
10326
10327
10328
10329
10330
10331
10332
10333
10334
10335
10336
10337
10338
10339
10340
10341
10342
10343
10344
10345
10346
10347
10348
10349
10350
10351
10352
10353
10354
10355
10356
10357
10358
10359
10360
10361
10362
10363
10364
10365
10366
10367
10368
10369
10370
10371
10372
10373
10374
10375
10376
10377
10378
10379
10380
10381
10382
10383
10384
10385
10386
10387
10388
10389
10390
10391
10392
10393
10394
10395
10396
10397
10398
10399
10400
10401
10402
10403
10404
10405
10406
10407
10408
10409
10410
10411
10412
10413
10414
10415
10416
10417
10418
10419
10420
10421
10422
10423
10424
10425
10426
10427
10428
10429
10430
10431
10432
10433
10434
10435
10436
10437
10438
10439
10440
10441
10442
10443
10444
10445
10446
1044

13770
13771
13772
13773
13774
13775
13776
13777
13778
13779
13780
13781
13782
13783
13784
13785
13786
13787
13788
13789
13790
13791
13792
13793
13794
13795
13796
13797
13798
13799
13800
13801
13802
13803
13804
13805
13806
13807
13808
13809
13810
13811
13812
13813
13814
13815
13816
13817
13818
13819
13820
13821
13822
13823
13824
13825
13826
13827
13828
13829
13830
13831
13832
13833
13834
13835
13836
13837
13838
13839
13840
13841
13842
13843
13844
13845
13846
13847
13848
13849
13850
13851
13852
13853
13854
13855
13856
13857
13858
13859
13860
13861
13862
13863
13864
13865
13866
13867
13868
13869
13870
13871
13872
13873
13874
13875
13876
13877
13878
13879
13880
13881
13882
13883
13884
13885
13886
13887
13888
13889
13890
13891
13892
13893
13894
13895
13896
13897
13898
13899
13900
13901
13902
13903
13904
13905
13906
13907
13908
13909
13910
13911
13912
13913
13914
13915
13916
13917
13918
13919
13920
13921
13922
13923
13924
13925
13926
13927
13928
13929
13930
13931
13932
13933
13934
13935
1393

...

Finding label for test item  17444
Finding label for test item  17445
Finding label for test item  17446
Finding label for test item  17447
Finding label for test item  17448
Finding label for test item  17449
Finding label for test item  17450
Finding label for test item  17451
Finding label for test item  17452
Finding label for test item  17453
Finding label for test item  17454
Finding label for test item  17455
Finding label for test item  17456
Finding label for test item  17457
Finding label for test item  17458
Finding label for test item  17459
Finding label for test item  17460
Finding label for test item  17461
Finding label for test item  17462
Finding label for test item  17463
Finding label for test item  17464
Finding label for test item  17465
Finding label for test item  17466
Finding label for test item  17467
Finding label for test item  17468
Finding label for test item  17469
Finding label for test item  17470
Finding label for test item  17471
Finding label for te

Finding label for test item  17679
Finding label for test item  17680
Finding label for test item  17681
Finding label for test item  17682
Finding label for test item  17683
Finding label for test item  17684
Finding label for test item  17685
Finding label for test item  17686
Finding label for test item  17687
Finding label for test item  17688
Finding label for test item  17689
Finding label for test item  17690
Finding label for test item  17691
Finding label for test item  17692
Finding label for test item  17693
Finding label for test item  17695
Finding label for test item  17694
Finding label for test item  17696
Finding label for test item  17697
Finding label for test item  17698
Finding label for test item  17699
Finding label for test item  17700
Finding label for test item  17701
Finding label for test item  17702
Finding label for test item  17703
Finding label for test item  17704
Finding label for test item  17705
Finding label for test item  17706
Finding label for te

Finding label for test item  17914
Finding label for test item  17915
Finding label for test item  17916
Finding label for test item  17917
Finding label for test item  17918
Finding label for test item  17919
Finding label for test item  17920
Finding label for test item  17921
Finding label for test item  17922
Finding label for test item  17923
Finding label for test item  17924
Finding label for test item  17925
Finding label for test item  17926
Finding label for test item  17927
Finding label for test item  17928
Finding label for test item  17929
Finding label for test item  17930
Finding label for test item  17931
Finding label for test item  17932
Finding label for test item  17933
Finding label for test item  17934
Finding label for test item  17935
Finding label for test item  17936
Finding label for test item  17937
Finding label for test item  17939
Finding label for test item  17938
Finding label for test item  17940
Finding label for test item  17942
Finding label for te

Finding label for test item  18149
Finding label for test item  18150
Finding label for test item  18152
Finding label for test item  18151
Finding label for test item  18153
Finding label for test item  18154
Finding label for test item  18155
Finding label for test item  18156
Finding label for test item  18157
Finding label for test item  18158
Finding label for test item  18159
Finding label for test item  18160
Finding label for test item  18161
Finding label for test item  18162
Finding label for test item  18163
Finding label for test item  18164
Finding label for test item  18165
Finding label for test item  18166
Finding label for test item  18167
Finding label for test item  18168
Finding label for test item  18169
Finding label for test item  18170
Finding label for test item  18171
Finding label for test item  18172
Finding label for test item  18173
Finding label for test item  18174
Finding label for test item  18175
Finding label for test item  18176
Finding label for te

Finding label for test item  18384
Finding label for test item  18385
Finding label for test item  18386
Finding label for test item  18387
Finding label for test item  18388
Finding label for test item  18389
Finding label for test item  18390
Finding label for test item  18391
Finding label for test item  18392
Finding label for test item  18393
Finding label for test item  18394
Finding label for test item  18395
Finding label for test item  18396
Finding label for test item  18397
Finding label for test item  18398
Finding label for test item  18399
Finding label for test item  18400
Finding label for test item  18401
Finding label for test item  18402
Finding label for test item  18403
Finding label for test item  18404
Finding label for test item  18405
Finding label for test item  18406
Finding label for test item  18407
Finding label for test item  18408
Finding label for test item  18409
Finding label for test item  18410
Finding label for test item  18411
Finding label for te

Write the results

In [34]:
# writing to .txt
fileName = "../../output/dhimalp-k" + str(k) + "-train-" + str(len(training_data)) + "-test-" + str(
    len(raw_test_data)) + "-" + str(int(time.time())) + ".dat"

np.savetxt(fileName, ratings_k25, fmt='%s')
print("INFO: Done writing results!")
print("Check ", fileName)

INFO: Done writing results!
Check  ../../output/dhimalp-k25-train-18497-test-18506-1582244248.dat


Score on Miner: 0.64

Done!